## Introduction

There are lottery systems in many games. The simplest implementation is that one gives a fixed probability and draws the lottery with that probability. The number of trials for a win has a geometric distribution. This distribution has a drawback that the variance is very large especially when the probability is low. So we'd like to have some alternative systems to solve this problem.

In [1]:
import numpy as np
import plotly
import plotly.graph_objs as go
import random

from utils import *

### Let's first see how the geometric distribution looks like

In [2]:
def geometric(p):
    """Returns the procedure for the geometric distribution"""
    def procedure():
        return random.random() < p
    return procedure

In [3]:
geometric(0.1)()

False

In [4]:
geometric_dic = mc(geometric(0.01), simulation=1_000_000)

In [5]:
get_mean_std(geometric_dic)

(98.88479034810126, 100.19243613568263)

In [6]:
def theoretical_geometric_mean_std(p):
    mean = 1 / p
    var = (1 - p) / p / p
    return mean, np.sqrt(var)

In [7]:
theoretical_geometric_mean_std(0.01)

(100.0, 99.498743710662)

In [8]:
plot_pdf(geometric_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '66fcc7d5-3d80-4f92-b8e2-d4b3c53c5ce5',
 …

In [9]:
plot_cdf(geometric_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '0413eb36-54f1-4d39-bbb5-aafaa7ec81f8',
 …

### For simple a geometric distribution, we see even though the mean is only 100, but there is still some probability to fail more than 300 trials

## A Better Solution

The challenging thing is how to make another probability distribution but keep the expectation unchanged. Suppose we'd like the expectation of every trial to be 0.01. When we fail for one trial, we don't want to lose completely but gain some value such as 0.005 which means in later trials it will generate the expectation of 0.005. 

Now we need to calculate the probability of winning and losing. Assume the winning probability is $a$, and the losing probability is $1 - a$. When win we gain 1, when lose we gain 0.005. So the "expectation" will be $a + 0.005 (1 - a)$, which should be equal to 0.01. So we can solve the equation to get that $a = \frac{1}{199}$.

So the initial probability is decreased, but later probability should raise, how? Take an example that we failed 60 trials, the gained value is accumulated to 0.3. If we win for the next trial, we consider that the value of 0.3 is used and it should be set back to zero, meaning that we only gain 0.7 since the gained value is decreased by 0.3. If the winning probability were still $\frac{1}{199}$, we gained less, so the probability should increase. The expectation of the next trial is $(1 - 0.3) a + 0.005 (1 - a)$. Again we can get the value of $a$.

One more thing to notice is that when the accumulated gained value is more than 0.99. The solution gives a probability that is greater than 1. In fact even if we have probability of 1 to win, the gain is still less than 0.01 if the accumulated gain is reset to zero after win. So in this case, we should not set the accumulated gain to zero, we should first add 0.01 and then subtract 1 instead.

### Let's see how to implement it in general

In [10]:
def accumulated(p, x, start=0):
    """Create a procedure that has the expectation of p and the gaining value increase by x for every failed trial"""
    assert p >= x >= 0
    left = start
    def procedure():
        nonlocal left
        if left + p < 1:
            # Note, p >= x and left + p < 1 guarantee 1 - left - x > 0
            r = (p - x) / (1 - left - x)
            win = random.random() < r
            if win:
                # Normally `left` should be set to 0.
                # Here we set to `start` to demonstrate the behavior when start from different values.
                # See Remark section
                left = start
            else:
                left += x
            return win
        else:
            left = left + p - 1
            return True
    return procedure

In [11]:
procedure = accumulated(0.01, 0.005)

In [12]:
acc_dic = mc(procedure, simulation=1_000_000)

In [13]:
get_mean_std(acc_dic)

(99.65935818218058, 57.38092788293979)

In [14]:
plot_pdf(acc_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'ffb134d6-4a99-4c6c-8a69-776ffb26405a',
 …

In [15]:
plot_cdf(acc_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '0bdcb05a-05fc-44e3-a3d5-c7bb0e0fa69f',
 …

### This seems to have a uniform distribution

In [16]:
procedure = accumulated(0.01, 0.003)

In [17]:
acc_dic = mc(procedure, simulation=1_000_000)

In [18]:
get_mean_std(acc_dic)

(99.25275488930805, 73.17226489650689)

In [19]:
plot_pdf(acc_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'a746014f-0b12-47f5-a619-a1bf5f997668',
 …

In [20]:
plot_cdf(acc_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'ea949260-41b8-4f8c-8b66-8dd5844d0065',
 …

In [21]:
procedure = accumulated(0.01, 0.007)

In [22]:
acc_dic = mc(procedure, simulation=1_000_000)

In [23]:
get_mean_std(acc_dic)

(100.47789389067525, 41.69663414751639)

In [24]:
plot_pdf(acc_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'ca84ff6b-5715-418b-b3fb-d74abe526180',
 …

In [25]:
plot_cdf(acc_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'a4198d39-568a-4370-b63a-f40d6b274c73',
 …

From the three examples above, we see the variance is reduce and you will never fail more than 400 times.

## Remark

In the above procedure, the accumulated gained value `left` is greater than or equal to 0.0. But it can also be negative without any alteration of the code. In that case, the winning probability is even smaller since it has to compensate the negative value to win.

In [26]:
procedure = accumulated(0.01, 0.005, start=-1.5)

In [27]:
acc_dic = mc(procedure, simulation=1_000_000)

In [28]:
get_mean_std(acc_dic)

(246.47843233916686, 143.7873340274906)

In [29]:
plot_pdf(acc_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '214080b1-1c70-4158-906c-679f5701dac8',
 …

In [30]:
plot_cdf(acc_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'ddf43009-d11b-40d8-9bae-b1ecc0140729',
 …

### Still the uniform distribution

## More complex methods

In the above method, when lose we gain a fixed value x. We can change it to make more complex methods. For example, gain x1 when the accumulated gain is less than 0.5 and gain x2 otherwise.

In [31]:
def accumulated_complex(p, x1, x2):
    assert p >= x1 >= 0 and p >= x2 >= 0
    left = 0
    def procedure():
        nonlocal left
        to_left = x1 if left < 0.5 else x2
        if left + p < 1:
            r = (p - to_left) / (1 - left - to_left)
            win = random.random() < r
            if win:
                left = 0
            else:
                left += to_left
            return win
        else:
            left = left + p - 1
            return True
    return procedure

In [32]:
procedure = accumulated_complex(0.01, 0.007, 0.003)

In [33]:
acc_dic = mc(procedure, simulation=1_000_000)

In [34]:
get_mean_std(acc_dic)

(100.22604250200482, 49.3414671281844)

In [35]:
plot_pdf(acc_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '554fe890-bfc2-49ed-a946-030584ab057a',
 …

In [36]:
plot_cdf(acc_dic)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '1cb185e7-3e5d-4ab4-92f1-83f908943785',
 …

## Conclusion

We introduced a lottery method that has lower variances than geometric distributions and can also easily keep the gaining expectation unchanged.